# Demonstration of Transfer Degree Roadmap Planning Capabilities 

Below we provide a demonstration of the capabilities of the system we have created for computing two-year to four-year transfer degree plans. These roadmaps detail how a student, by following the plan, would earn an assocaites degree at a two-year college, followed by a bachelor's degree at a four-year university.

What is demonstarted here is essentially what is being computed behind the scenes in the tranfer portal (e.g., https://transfer.kydegreeplans.com) when a student uses the "2 Year to 4 Year Plan" option.

## Overview
At a high level, the code below performs the following steps. 
1. A user selects the source and destination schools, as well as the programs at the source and destination schools that the student would like to complete. In this notebook, we will specify them by assigning values to source_program and destination_program variables, but in the web application, these are selected from pulldown menus.
2. The data required to compute the transfer plan, that has been previously stored in the Blackbriar data system at http://kydegreeplans.com, is extracted through API function calls. The following information must be extracted and formatted for use by the Curricular Analytics engine:
 - The course catalogs from both the source and destination schools,
 - Yiming describe remaining data pulled
 
3. This data is supplied to a highly sophisticated optimization algorithm, which makes use of the Gurobi commerical solver, in order to compute a transfer plan that is optimized around the particular set of programs that the user has selected.  The details of this optimization algorithm are described in more detail below.

The following Julia programming language packages are required in this demonstration.  Note the extensive use of the Curricular Analytics toolbox capabilites.

In [1]:
using JuMP, Gurobi  # optimization packages
using CSV, DataFrames, JSON, DataStructures # data handling packages
using CurricularAnalytics, CurricularOptimization, CurricularVisualization # curricular analytics toolbox packages

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-23


WebIO._IJuliaInit()

## Create Course Catalogs
This function will be used to add coruses to the course catalogs required by this application. This function is a part of the Curricular Analytics capabilites, and woudl normally be called via API, but for the purpose of this demonstration, it must be deifned here.

In [2]:
function add_courses_to_catalog(catalog, json_courses, destinationSchool)
    for course in json_courses
        c_split = split(course["code"], " ")
        prefix = c_split[1]
        num = c_split[2]
        if num === nothing
            continue
        end
        cid = mod(hash(course["name"] * prefix * num * destinationSchool), UInt32)
        cid = convert(Int, cid)
        course = Course(course["name"], course["maximum_credits"], prefix=prefix, num=num, institution=destinationSchool, id=cid)
        add_course!(catalog, course)
    end
end

add_courses_to_catalog (generic function with 1 method)

The following code reads the course catalogs, which have been extracted from the Blackbriar data system as JSON files. In addition, we select the source and destination programs.  In this case we are selecting a source program from KCTCS, so we must read in that catalog, and a destination program from Eastern Kentucky Univeristy, so we must read in that catalog. These catalogs are constructed by reading in all of the course information available from these schools in the Blackbriar data system, and available at http://kydegreeplans.com.

In [3]:
#include("/Users/heileman/.julia/dev/CurricularOptimization/src/TransferArticulation.jl");
include("./Conversions.jl");

source_program = "Associate of Art"
destination_program = "Computing System"

university_courses = Dict{String,Any}();
university_catalogs = Dict{String,Any}();
kctcs_courses = JSON.parsefile("./bb_data/kctcs_courses.json");
kctcs_reqs = JSON.parsefile("./bb_data/Kentucky_college_programs/$(source_program).json");
eku_courses = JSON.parsefile("./bb_data/eku_courses.json");
eku_reqs = JSON.parsefile("./bb_data/Kentucky_university_programs/$(destination_program).json");

In [4]:
source_school = "KCTCS" # name of the destination school
source_school_catalog = CourseCatalog(source_school, source_school);
add_courses_to_catalog(source_school_catalog, kctcs_courses, source_school);

source_requirements = parse_requirement_set(kctcs_reqs, source_school_catalog);
source_requirements = CurricularOptimization.individualize_set_id(source_requirements);
flat_reqs_source = flatten_requirements(source_requirements);

destination_school = "EKU"
destination_school_catalog = CourseCatalog(destination_school, destination_school);
add_courses_to_catalog(destination_school_catalog, eku_courses, destination_school);

destination_requirements_original = parse_requirement_set(eku_reqs, destination_school_catalog);
destination_requirements = deepcopy(destination_requirements_original)
destination_requirements = CurricularOptimization.individualize_set_id(destination_requirements);
flat_reqs_destination = flatten_requirements(destination_requirements);

In [5]:
transfer_map_db = CSV.read("./bb_data/equivs.csv", DataFrame)
transfer_map_db = clean_transfer_file(
    transfer_map_db, source_school_catalog, destination_school_catalog
)

transfer_map, reverse_transfer_map = CurricularOptimization.transfer_mapper(
    transfer_map_db, source_school_catalog, destination_school_catalog
)

course_id_array_source, reverse_course_id_map_source = CurricularOptimization.get_course_ids(source_school_catalog)
course_id_array_destination, reverse_course_id_map_destination = CurricularOptimization.get_course_ids(
    destination_school_catalog
)

time_hours = 0.01;
num_solutions = 1;

# requisite_destination = DataFrame()
requisite_source = DataFrame()

### run optimization with min_credit_sum objective function
course_to_requirement_source,
curriculum_source,
transferred_curriculum,
course_to_requirement_destination,
last_2_year_curriculum = 
CurricularOptimization.optimize_plan(source_requirements, destination_requirements_original, source_school_catalog, destination_school_catalog, 
transfer_map, reverse_transfer_map, course_id_array_source, reverse_course_id_map_source, course_id_array_destination, reverse_course_id_map_destination;
requisite_source=requisite_source,
time_limit=30, num_solutions=num_solutions)



thread = 1 warning: only found 77 / 78 columns around data row: 617. Filling remaining columns with `missing`
Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-23
Set parameter TimeLimit to value 30
Set parameter PoolSolutions to value 1
Set parameter PoolSearchMode to value 2
Set parameter PoolGap to value 0
The root id is 18
The root id is 5
Set parameter PoolGap to value 0
Set parameter PoolSearchMode to value 2
Set parameter PoolSolutions to value 1
Set parameter TimeLimit to value 30
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-5700HQ CPU @ 2.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4718 rows, 142996 columns and 238596 nonzeros
Model fingerprint: 0xad3596fb
Model has 7725 general constraints
Variable types: 0 continuous, 142996 integer (142996 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e

(2149×38 DataFrame. Omitted printing of 29 columns
│ Row  │ x1    │ x2    │ x3    │ x4    │ x5    │ x6    │ x7    │ x8    │ x9    │
│      │ Int64 │ Int64 │ Int64 │ Int64 │ Int64 │ Int64 │ Int64 │ Int64 │ Int64 │
├──────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┤
│ 1    │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │
│ 2    │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │
│ 3    │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │
│ 4    │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │
│ 5    │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │
│ 6    │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │
│ 7    │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │
│ 8    │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │
│ 9    │ 0     │ 0     │ 0     │ 0     │ 0     │ 0     │ 0

In [6]:
source_courses_list = [v for (k, v) in source_school_catalog.catalog];
# Updates 0 credits courses
for course in source_courses_list
    course.credit_hours > 99 ? course.credit_hours = 0.0 : nothing
end

destination_courses_list = [v for (k, v) in destination_school_catalog.catalog];
# updates 0 credit courses
for course in destination_courses_list
    course.credit_hours > 99 ? course.credit_hours = 0.0 : nothing
end

# save the transfer plan report from the output of the optimize_plan function
source_requirements_credit, destination_requirements_credit = floor(source_requirements.credit_hours), floor(destination_requirements_original.credit_hours)
CurricularOptimization.transfer_plan_report(curriculum_source, course_to_requirement_source, transferred_curriculum, last_2_year_curriculum, course_to_requirement_destination, 
    source_courses_list, flat_reqs_source, destination_courses_list, flat_reqs_destination,
    source_program, source_school, destination_program, destination_school,
    source_requirements_credit, destination_requirements_credit,
    reverse_transfer_map, source_school_catalog, destination_school_catalog)

Transfer Analysis: KCTCS Associate of Art Program --> EKU Computing System Program: 
62.0 + 79.0 = 141.0 total credit hours
KCTCS credits: 
    62.0 credits required for a 61.0 credit hour program
Transfer credits:
    47.0 credit hours transfer
    15.0 credit hours lost
EKU credits: 
    79.0 additional credits for a 120.0 credit hour program

The following courses will earn the Degree in Associate of Art program at KCTCS: 
Total credit hours = 62.0
    ART 100 Introduction to Art (Credits: 3.0):>>>>Transferrable
    satisfies Arts & Humanities - Humanities 
------------------------------
    AST 101 Frontiers of Astronomy (Credits: 3.0):>>>>Not Transferrable
    satisfies Natural Sciences CourseSet 
------------------------------
    BIO 148 Introductory Biology I (Credits: 3.0):>>>>Transferrable
    satisfies Foreign Language 
------------------------------
    CHE 145 Introductory General Chemistry Laboratory (Credits: 1.0):>>>>Transferrable
    satisfies Lab 
--------------------

In [7]:
# Create curriculum for source school
curriculum_source = convert(Matrix{Int}, curriculum_source)
two_year_course_list = Array{Course,1}()
for (i, value) in enumerate(curriculum_source)
    if value > 0.5
        push!(two_year_course_list, source_courses_list[i])
    end
end

source_curriculum = CurricularAnalytics.Curriculum("Two Year", two_year_course_list)
source_degreeplan = CurricularOptimization.optimize_plan(source_curriculum, 4, 8, 25, [CurricularOptimization.balance_obj])
visualize(source_degreeplan, notebook=true)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-23
An optimal solution was found with objective value = 8.0


WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol, Any}(:src => "https://curricula-api-embed.damoursystems.com/", :id => "curriculum", :events => Dict{Symbol, WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"Two Year\",\"dp_name\":\"\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"ECO 201\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Principles of Microeconomics\",\"id\":423308936,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"IEC 291\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"IECE Practicum/Cooperative Education\",\"id\":190471220,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"IEC 235\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Introduction to Inclusive Education\",\"id\":286853854,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"BIO 148\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Introductory Biology I\",\"id\":502346721,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"IEC 260\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Infant and Toddler Education and Programming\",\"id\":3846055305,\"credits\":3.0,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"HIS 101\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"World Civilization I\",\"id\":1391597362,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CHE 170\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"General College Chemistry I\",\"id\":2759835203,\"credits\":4.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CRJ 279\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Terrorism and Political Violence\",\"id\":1728467358,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"FAM 253\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Human Sexuality:  Development, Behavior, and Attitudes\",\"id\":3976753628,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"COM 252\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Introduction to Interpersonal Communication\",\"id\":4139891793,\"credits\":3.0,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"ENG 101\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Writing I\",\"id\":4203010242,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"GEO 152\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Regional Geography of the World\",\"id\":1007206674,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CHE 145\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Introductory General Chemistry Laboratory\",\"id\":1119549100,\"credits\":1.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"REL 101\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Introduction to Religio

In [8]:
# Create curriculum for destination school
last_2_year_curriculum = convert(Matrix{Int}, last_2_year_curriculum)
four_year_course_list = Array{Course,1}()
for (i, value) in enumerate(last_2_year_curriculum)
    if value > 0.5
        push!(four_year_course_list, destination_courses_list[i])
    end
end



In [9]:
dest_curriculum = CurricularAnalytics.Curriculum("Four Year", four_year_course_list)
dest_degreeplan = CurricularOptimization.optimize_plan(dest_curriculum, 4, 8, 25, [CurricularOptimization.balance_obj])
visualize(dest_degreeplan, notebook=true)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-23
An optimal solution was found with objective value = 12.0


WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol, Any}(:src => "https://curricula-api-embed.damoursystems.com/", :id => "curriculum", :events => Dict{Symbol, WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"Four Year\",\"dp_name\":\"\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"QMB 200\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Business Statistics I\",\"id\":2743475061,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"ENG 102\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Research, Writing, & Rhetoric\",\"id\":2292047404,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CMS 210\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Public Speaking\",\"id\":846014856,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CIS 340\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Electronic Business Technologies and Tools.\",\"id\":640031206,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"BTS 300\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Professional Skills Seminar\",\"id\":1705147435,\"credits\":0.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"MGT 300\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Principles of Management\",\"id\":2354522378,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"BTS 400\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"College to Careers Seminar\",\"id\":1387784944,\"credits\":0.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CIS 240\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Intro to Web Info Systems\",\"id\":1752176207,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"BTO 100\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Student Success Seminar\",\"id\":1821868359,\"credits\":1.0,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"GBU 480\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Business Strategy\",\"id\":3919091473,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"FIN 300\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Business Finance I\",\"id\":4004630018,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CCT 300W\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Managerial Reports\",\"id\":1182524942,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CIS 335\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Data Base Management\",\"id\":1033217435,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CIS 380\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Information Sys Analysis & Des\",\"id\":3358245624,\"credits\":3.0,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"MAT 112B\",\"metrics\":{\"centrality\":0,\"complexity\":1